In [34]:
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.acquisition import gaussian_ei, gaussian_pi
from sklearn.preprocessing import StandardScaler
import warnings
from sklearn.exceptions import ConvergenceWarning
import itertools as it

# Suppress specific warnings
warnings.filterwarnings('ignore', category=ConvergenceWarning)


### 3 variables

In [25]:

# Define functions
def noisy_function(x):
    return np.sin(10 * np.pi * x[0]) * np.cos(10 * np.pi * x[1]) + np.random.normal(0, 0.1)

#def contamination_function(x):
#    noise = np.random.normal(0, 0.1)
#    dist1 = np.sqrt((x[0] - 0.3)**2 + (x[1] - 0.3)**2 + (x[2] - 0.3)**2)
#    dist2 = np.sqrt((x[0] - 0.7)**2 + (x[1] - 0.7)**2 + (x[2] - 0.7)**2)
#    if dist1 < 0.3:  # Increase the radius of the high-value region
#        return 10 * (1 - dist1 / 0.3) + noise  # Adjust the scaling factor
#    elif dist2 < 0.3:  # Increase the radius of the second high-value region
#        return 5 * (1 - dist2 / 0.3) + noise  # Adjust the scaling factor
#    else:
#        return noise

def contamination_function(x):
    noise = np.random.normal(0, 0.1)
    dist1 = np.sqrt((x[0] - 0.3)**2 + (x[1] - 0.3)**2 + (x[2] - 0.3)**2)
    dist2 = np.sqrt((x[0] - 0.7)**2 + (x[1] - 0.7)**2 + (x[2] - 0.7)**2)
    if dist1 < 0.1:
        return 10 * (1 - dist1) + noise
    elif dist2 < 0.1:
        return 5 * (1 - dist2) + noise
    else:
        return noise
    
def complex_function(x):
    return np.exp(-((x[0] - 0.5) ** 2 + (x[1] - 0.5) ** 2 + (x[2] - 0.5) ** 2) / 0.1) * np.sin(5 * np.pi * x[2])

# Function to generate grid
def generate_grid(ranges, num_points):
    grid_axes = [np.linspace(start, end, num_points) for start, end in ranges]
    grid = np.array(list(it.product(*grid_axes)))
    return grid


In [21]:

# Define search space and grid
input_ranges = [(0.0, 1.0), (0.0, 1.0), (0.0, 1.0)]
GG = 150  # Number of data points for each variable in the grid
X_grid = generate_grid(input_ranges, GG)
X_grid_df = pd.DataFrame(X_grid, columns=[f'Input_{i+1}' for i in range(3)])

# Function to perform Bayesian Optimization
def perform_bayesian_optimization(func, acquisition_strategy, X_initial, y_initial, max_iter=50):
    results_df = pd.DataFrame()

    # Convert initial samples to DataFrame
    X = pd.DataFrame(X_initial, columns=[f'Input_{i+1}' for i in range(3)])
    y = pd.DataFrame(y_initial, columns=['Output'])

    # Standardize inputs and outputs
    scaler_X = StandardScaler()
    X_scaled = scaler_X.fit_transform(X)
    scaler_y = StandardScaler()
    y_scaled = scaler_y.fit_transform(y)

    # Define the search space for Bayesian Optimization
    space = [
        Real(1e-3, 0.5, name='length_scale'),
        Real(1e-12, 1e-2, name='noise_level')
    ]

    # Evaluation function (Log-Likelihood)
    def evaluate_model(length_scale, noise_level):
        kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=length_scale) + WhiteKernel(noise_level=noise_level)
        GPR_model = GaussianProcessRegressor(kernel=kernel, alpha=noise_level)
        GPR_model.fit(X_scaled, y_scaled)
        log_likelihood = GPR_model.log_marginal_likelihood()
        return -log_likelihood

    # Use the search space in the objective function
    @use_named_args(space)
    def objective(**params):
        return evaluate_model(**params)

    # Perform Bayesian Optimization
    res = gp_minimize(objective, space, n_calls=50, n_initial_points=10, random_state=32)

    # Get the best parameters
    best_length_scale = res.x[0]
    best_noise_level = res.x[1]

    # Define the best kernel
    best_kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=best_length_scale) + WhiteKernel(noise_level=best_noise_level)

    # Fit the Gaussian Process with the best parameters
    GPR_model_best = GaussianProcessRegressor(kernel=best_kernel, alpha=best_noise_level)
    GPR_model_best.fit(X_scaled, y_scaled)

    # Perform the iterative optimization
    for iteration in range(max_iter):
        # Generate grid for predictions
        X_grid_scaled = scaler_X.transform(X_grid_df)
        mean_scaled, std_scaled = GPR_model_best.predict(X_grid_scaled, return_std=True)
        mean_scaled = mean_scaled.reshape(-1, 1)  # Reshape to 2D array
        mean = scaler_y.inverse_transform(mean_scaled).flatten()  # Inverse transform to original scale and flatten to 1D array
        std = std_scaled * scaler_y.scale_[0]  # Scale standard deviation appropriately

        # Acquisition function
        if acquisition_strategy == 'UCB_fixed':
            beta = 2.6  # Fixed beta strategy
            acquisition_function = mean + beta * std
        elif acquisition_strategy == 'UCB_decreasing':
            if iteration < max_iter // 4:
                beta = 2.6
            elif iteration < max_iter // 2:
                beta = 1.3
            elif iteration < 3 * max_iter // 4:
                beta = 0.5
            else:
                beta = 0.01  # Decreasing beta strategy
            acquisition_function = mean + beta * std
        elif acquisition_strategy == 'UCB_decreasing_larger':
            if iteration < max_iter // 4:
                beta = 4
            elif iteration < max_iter // 2:
                beta = 2.9
            elif iteration < 3 * max_iter // 4:
                beta = 1.5
            else:
                beta = 0.5  # Decreasing beta strategy
            acquisition_function = mean + beta * std
        elif acquisition_strategy == 'EI_fixed':
            xi = 0.01  # Fixed xi strategy
            acquisition_function = gaussian_ei(X_grid_scaled, model=GPR_model_best, xi=xi)
        elif acquisition_strategy == 'EI_changing':
            if iteration < max_iter // 4:
                xi = 0.01
            elif iteration < max_iter // 2:
                xi = 0.005
            elif iteration < 3 * max_iter // 4:
                xi = 0.002
            else:
                xi = 0.001  # Changing xi strategy
            acquisition_function = gaussian_ei(X_grid_scaled, model=GPR_model_best, xi=xi)
        elif acquisition_strategy == 'PI_fixed':
            xi = 0.01  # Fixed xi strategy
            acquisition_function = gaussian_pi(X_grid_scaled, model=GPR_model_best, xi=xi)
        elif acquisition_strategy == 'PI_changing':
            if iteration < max_iter // 4:
                xi = 0.01
            elif iteration < max_iter // 2:
                xi = 0.005
            elif iteration < 3 * max_iter // 4:
                xi = 0.002
            else:
                xi = 0.001  # Changing xi strategy
            acquisition_function = gaussian_pi(X_grid_scaled, model=GPR_model_best, xi=xi)
        else:
            raise ValueError("Unknown acquisition strategy")

        # Find the next query point
        idx_max = np.argmax(acquisition_function)
        next_query = X_grid[idx_max]
        next_query_output = func(next_query)

        # Append new data
        new_data = pd.DataFrame([next_query], columns=[f'Input_{i+1}' for i in range(3)])
        new_data['Output'] = next_query_output
        X = pd.concat([X, new_data.drop(columns=['Output'])])
        y = pd.concat([y, new_data[['Output']]])

        # Update the model
        X_scaled = scaler_X.fit_transform(X)
        y_scaled = scaler_y.fit_transform(y)
        GPR_model_best.fit(X_scaled, y_scaled)

        # Record the result
        results_df.loc[iteration, f'{func.__name__}_{acquisition_strategy}'] = next_query_output
        print(f"Finished iteration {iteration} at {acquisition_strategy}")

    return results_df

# Run the optimization for a given function and acquisition strategies
def run_simulation_for_function(func):
    acquisition_strategies = ['UCB_fixed', 'UCB_decreasing', 'UCB_decreasing_larger']
    all_results_df = pd.DataFrame()

    # Generate initial samples (common for all strategies)
    X_initial = np.random.uniform(0, 1, size=(10, 3))
    y_initial = np.array([func(x) for x in X_initial])

    for i, strategy in enumerate(acquisition_strategies):
        results_df = perform_bayesian_optimization(func, strategy, X_initial, y_initial)
        all_results_df = pd.concat([all_results_df, results_df], axis=1)
        print(f"Finished process {i + 1} out of {len(acquisition_strategies)}")

    return all_results_df

# Example usage:
chosen_function = contamination_function  # Change this to focus on a different function (noisy, complex, contamination)
results = run_simulation_for_function(chosen_function)

# Print the results
print(results)

# Calculate and print the real maximum of the function over the grid
real_maximum = np.max([chosen_function(x) for x in X_grid])
print(f"Real maximum of the function: {real_maximum}")

Finished iteration 0 at UCB_fixed
Finished iteration 1 at UCB_fixed
Finished iteration 2 at UCB_fixed
Finished iteration 3 at UCB_fixed
Finished iteration 4 at UCB_fixed
Finished iteration 5 at UCB_fixed
Finished iteration 6 at UCB_fixed
Finished iteration 7 at UCB_fixed
Finished iteration 8 at UCB_fixed
Finished iteration 9 at UCB_fixed
Finished iteration 10 at UCB_fixed
Finished iteration 11 at UCB_fixed
Finished iteration 12 at UCB_fixed
Finished iteration 13 at UCB_fixed
Finished iteration 14 at UCB_fixed
Finished iteration 15 at UCB_fixed
Finished iteration 16 at UCB_fixed
Finished iteration 17 at UCB_fixed
Finished iteration 18 at UCB_fixed
Finished iteration 19 at UCB_fixed
Finished iteration 20 at UCB_fixed
Finished iteration 21 at UCB_fixed
Finished iteration 22 at UCB_fixed
Finished iteration 23 at UCB_fixed
Finished iteration 24 at UCB_fixed
Finished iteration 25 at UCB_fixed
Finished iteration 26 at UCB_fixed
Finished iteration 27 at UCB_fixed
Finished iteration 28 at UCB_f

Real maximum of the function: 10.15598175508433


## 2 variables

In [48]:
# Define functions for 2 inputs
def noisy_function2(x):
    return np.sin(10 * np.pi * x[0]) * np.cos(10 * np.pi * x[1]) + np.random.normal(0, 0.1)

def contamination_function2(x):
    noise = np.random.normal(0, 0.1)
    dist1 = np.sqrt((x[0] - 0.3)**2 + (x[1] - 0.3)**2)
    dist2 = np.sqrt((x[0] - 0.7)**2 + (x[1] - 0.7)**2)
    if dist1 < 0.05:  # Increase the radius of the high-value region
        return 10 * (1 - dist1 / 0.3) + noise  # Adjust the scaling factor
    elif dist2 < 0.05:  # Increase the radius of the second high-value region
        return 5 * (1 - dist2 / 0.3) + noise  # Adjust the scaling factor
    else:
        return noise

def complex_function2(x):
    return np.exp(-((x[0] - 0.5) ** 2 + (x[1] - 0.5) ** 2) / 0.1) * np.sin(5 * np.pi * x[1])




# Function to generate grid
def generate_grid(ranges, num_points):
    grid_axes = [np.linspace(start, end, num_points) for start, end in ranges]
    grid = np.array(list(it.product(*grid_axes)))
    return grid


In [50]:

# Define search space and grid
input_ranges = [(0.0, 1.0), (0.0, 1.0)]
GG = 1000  # Number of data points for each variable in the grid
X_grid = generate_grid(input_ranges, GG)
X_grid_df = pd.DataFrame(X_grid, columns=[f'Input_{i+1}' for i in range(2)])

# Function to perform Bayesian Optimization  ***********************************************************
def perform_bayesian_optimization(func, acquisition_strategy, X_initial, y_initial, max_iter=10):
    results_df = pd.DataFrame()

    # Convert initial samples to DataFrame
    X = pd.DataFrame(X_initial, columns=[f'Input_{i+1}' for i in range(2)])
    y = pd.DataFrame(y_initial, columns=['Output'])

    # Standardize inputs and outputs
    scaler_X = StandardScaler()
    X_scaled = scaler_X.fit_transform(X)
    scaler_y = StandardScaler()
    y_scaled = scaler_y.fit_transform(y)

    # Define the search space for Bayesian Optimization
    space = [
        Real(1e-3, 0.5, name='length_scale'),
        Real(1e-12, 1e-2, name='noise_level')
    ]

    # Evaluation function (Log-Likelihood)
    def evaluate_model(length_scale, noise_level):
        kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=length_scale) + WhiteKernel(noise_level=noise_level)
        GPR_model = GaussianProcessRegressor(kernel=kernel, alpha=noise_level)
        GPR_model.fit(X_scaled, y_scaled)
        log_likelihood = GPR_model.log_marginal_likelihood()
        return -log_likelihood

    # Use the search space in the objective function
    @use_named_args(space)
    def objective(**params):
        return evaluate_model(**params)

    # Perform Bayesian Optimization*********************************************************************** 
    res = gp_minimize(objective, space, n_calls=50, n_initial_points=25, random_state=32)

    # Get the best parameters
    best_length_scale = res.x[0]
    best_noise_level = res.x[1]

    # Define the best kernel
    best_kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=best_length_scale) + WhiteKernel(noise_level=best_noise_level)

    # Fit the Gaussian Process with the best parameters
    GPR_model_best = GaussianProcessRegressor(kernel=best_kernel, alpha=best_noise_level)
    GPR_model_best.fit(X_scaled, y_scaled)

    # Perform the iterative optimization
    for iteration in range(max_iter):
        # Generate grid for predictions
        X_grid_scaled = scaler_X.transform(X_grid_df)
        mean_scaled, std_scaled = GPR_model_best.predict(X_grid_scaled, return_std=True)
        mean_scaled = mean_scaled.reshape(-1, 1)  # Reshape to 2D array
        mean = scaler_y.inverse_transform(mean_scaled).flatten()  # Inverse transform to original scale and flatten to 1D array
        std = std_scaled * scaler_y.scale_[0]  # Scale standard deviation appropriately

        # Acquisition function
        if acquisition_strategy == 'UCB_fixed':
            beta = 1.5  # Fixed beta strategy
            acquisition_function = mean + beta * std
        elif acquisition_strategy == 'UCB_decreasing':
            if iteration < max_iter // 4:
                beta = 1.96
            elif iteration < max_iter // 2:
                beta = 1
            elif iteration < 3 * max_iter // 4:
                beta = 0.5
            else:
                beta = 0.01  # Decreasing beta strategy
            acquisition_function = mean + beta * std
        elif acquisition_strategy == 'UCB_decreasing_larger':
            if iteration < max_iter // 4:
                beta = 1.96
            elif iteration < max_iter // 2:
                beta = 4
            elif iteration < 3 * max_iter // 4:
                beta = 1
            else:
                beta = 0.01  # Decreasing beta strategy
            acquisition_function = mean + beta * std
        elif acquisition_strategy == 'EI_fixed':
            xi = 0.01  # Fixed xi strategy
            acquisition_function = gaussian_ei(X_grid_scaled, model=GPR_model_best, xi=xi)
        elif acquisition_strategy == 'EI_changing':
            if iteration < max_iter // 4:
                xi = 0.01
            elif iteration < max_iter // 2:
                xi = 0.005
            elif iteration < 3 * max_iter // 4:
                xi = 0.002
            else:
                xi = 0.001  # Changing xi strategy
            acquisition_function = gaussian_ei(X_grid_scaled, model=GPR_model_best, xi=xi)
        elif acquisition_strategy == 'PI_fixed':
            xi = 0.01  # Fixed xi strategy
            acquisition_function = gaussian_pi(X_grid_scaled, model=GPR_model_best, xi=xi)
        elif acquisition_strategy == 'PI_changing':
            if iteration < max_iter // 4:
                xi = 0.01
            elif iteration < max_iter // 2:
                xi = 0.005
            elif iteration < 3 * max_iter // 4:
                xi = 0.002
            else:
                xi = 0.001  # Changing xi strategy
            acquisition_function = gaussian_pi(X_grid_scaled, model=GPR_model_best, xi=xi)
        else:
            raise ValueError("Unknown acquisition strategy")

        # Find the next query point
        idx_max = np.argmax(acquisition_function)
        next_query = X_grid[idx_max]
        next_query_output = func(next_query)

        # Append new data
        new_data = pd.DataFrame([next_query], columns=[f'Input_{i+1}' for i in range(2)])
        new_data['Output'] = next_query_output
        X = pd.concat([X, new_data.drop(columns=['Output'])])
        y = pd.concat([y, new_data[['Output']]])

        # Update the model
        X_scaled = scaler_X.fit_transform(X)
        y_scaled = scaler_y.fit_transform(y)
        GPR_model_best.fit(X_scaled, y_scaled)

        # Record the result
        results_df.loc[iteration, f'{func.__name__}_{acquisition_strategy}'] = next_query_output
        #print(f"Finished iteration {iteration} at {acquisition_strategy}")

    return results_df

# Run the optimization for a given function and acquisition strategies
def run_simulation_for_function(func):
    acquisition_strategies = ['UCB_fixed', 'UCB_decreasing', 'UCB_decreasing_larger']
    all_results_df = pd.DataFrame()

    # Generate initial samples (common for all strategies)
    X_initial = np.random.uniform(0, 1, size=(10, 2))
    y_initial = np.array([func(x) for x in X_initial])

    for i, strategy in enumerate(acquisition_strategies):
        results_df = perform_bayesian_optimization(func, strategy, X_initial, y_initial)
        all_results_df = pd.concat([all_results_df, results_df], axis=1)
        print(f"Finished process {i + 1} out of {len(acquisition_strategies)}")

    return all_results_df

# Example usage:
chosen_function = contamination_function2  # Change this to focus on a different function (noisy, complex, contamination)
results = run_simulation_for_function(chosen_function)

# Print the results
print(results)

# Calculate and print the real maximum of the function over the grid
real_maximum = np.max([chosen_function(x) for x in X_grid])
print(f"Real maximum of the function: {real_maximum}")

Finished process 1 out of 3
Finished process 2 out of 3
Finished process 3 out of 3
   contamination_function2_UCB_fixed  contamination_function2_UCB_decreasing  \
0                          -0.054249                                0.133706   
1                           0.223626                                0.113096   
2                          -0.013112                               -0.019527   
3                           0.004233                               -0.038643   
4                          -0.035318                               -0.080171   
5                          -0.089687                                0.004058   
6                           0.146594                                0.019926   
7                          -0.040821                                0.069506   
8                           0.039429                                0.065903   
9                          -0.036009                                0.077123   

   contamination_function2_UCB_decr